# Import the relevant packages

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

## download MNIST datasate

In [3]:
#tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
#with_info = True : provide a tuple containing info about cersion, feature and number of sample
#as_supervised  =True: load data in 2-tuple structure [input, target]
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)


## Preprocessed data

In [62]:
#Extract the train and test data
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

#Validation dataset taken from train data set. We take 10% of train data test
#setting the number of validation sample
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

#convert number of validation sample to integer
num_validation_samples =tf.cast(num_validation_samples, tf.int64)

#Store number of test sample in dedicated variable & cast it to integer
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#Sacle the date into inpute between 0 and 1
#define function that scale the input
def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

#Apllied our scale function to map function
#Scale our train and validation data
scaled_train_and_validation_data  =mnist_train.map(scale)

#Scale our test data
test_data = mnist_test.map(scale)

#Suffling validation and train data
#Defining Buffer size, this is a hyperparameter so we can explore it to fine tuning our model
BUFFER_SIZE = 10000

suffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#Create Validation Data
validation_data = suffled_train_and_validation_data.take(num_validation_samples)

#Create Train Data
train_data = suffled_train_and_validation_data.skip(num_validation_samples)

#Set batch size
BATCH_SIZE = 150

#Set batch for train, validation and test data
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

#Make Validation data has a same shape and information as train and test data
validation_inputs, validation_outputs = next(iter(validation_data))

# Model

## outline the model

In [63]:
#Declare input, output and hidden layer size
input_size = 784
output_size = 10
hidden_layer_size = 5000

#Outline the model
model = tf.keras.Sequential([
        #Firt layer or input layer
        #each observation is 28x28x1 pixeles, it is a tensor of rank 3
        #Flatten the image to a vector with .Flattern method
                            tf.keras.layers.Flatten(input_shape=(28,28,1)), #input Layer
        #Next step is define hidden layer
        #This argmunet is used to implementing output activation(dot(input, weight) + bias)
        #We use activation of relu
                            tf.keras.layers.Dense(hidden_layer_size, activation ='relu'), #1st hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation ='relu'), #2nd hidden layer
        #the last it Output Layer with softmax activation
                            tf.keras.layers.Dense(output_size, activation ='softmax')
                            ])

## Choose epitimizer and loss function

In [64]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Training

In [65]:
#Define number of epoch that will be used
NUM_EPOCHS = 10

#Train the model
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_outputs), verbose = 2)

Epoch 1/10
360/360 - 64s - loss: 0.1979 - accuracy: 0.9395 - val_loss: 0.1565 - val_accuracy: 0.9532 - 64s/epoch - 179ms/step
Epoch 2/10
360/360 - 65s - loss: 0.0791 - accuracy: 0.9756 - val_loss: 0.1007 - val_accuracy: 0.9680 - 65s/epoch - 180ms/step
Epoch 3/10
360/360 - 67s - loss: 0.0535 - accuracy: 0.9828 - val_loss: 0.0498 - val_accuracy: 0.9843 - 67s/epoch - 185ms/step
Epoch 4/10
360/360 - 76s - loss: 0.0425 - accuracy: 0.9866 - val_loss: 0.0398 - val_accuracy: 0.9883 - 76s/epoch - 211ms/step
Epoch 5/10
360/360 - 75s - loss: 0.0326 - accuracy: 0.9898 - val_loss: 0.0387 - val_accuracy: 0.9882 - 75s/epoch - 208ms/step
Epoch 6/10
360/360 - 75s - loss: 0.0306 - accuracy: 0.9904 - val_loss: 0.0329 - val_accuracy: 0.9898 - 75s/epoch - 209ms/step
Epoch 7/10
360/360 - 72s - loss: 0.0267 - accuracy: 0.9913 - val_loss: 0.0186 - val_accuracy: 0.9932 - 72s/epoch - 199ms/step
Epoch 8/10
360/360 - 66s - loss: 0.0251 - accuracy: 0.9926 - val_loss: 0.0337 - val_accuracy: 0.9915 - 66s/epoch - 183

# Testing 

In [66]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test Loss: {0:2f}. Test Accuracy: {1:2f}%'.format(test_loss, test_accuracy * 100.))

1/1 [==============================] - 2s 2s/step - loss: 0.0980 - accuracy: 0.9807
Test Loss: 0.098018. Test Accuracy: 98.070002%
